# Crawling Data

*Crawling data* adalah salah satu proses dalam mengumpulkan sebuah data, dilakukan dengan cara mengambil data atau informasi yang berasal dari sebuah webstie atau berbagai sumber dari internet seperti website berita, artikel, twitter dan masih banyak lainnya.


Dalam *Crawling Data* terdapat sebuah library yang sering digunakan dalam proses *Crawling Data* yakni **Scrapy**. Library **Scrapy** ini memiliki banyak fungsi salah satunya yakni mengambil data kemudian memprosesnya hingga data tersebut kemudian disimpan dengan skala yang besar.

# Tahapan dalam Crawling Data

Adapun beberapa tahapan yang perlu dilakukan dalam *Crawling Data* yakni :
1. Instalasi Scrapy
2. Import Library
3. Program Untuk *Crawling*

# 1. Instalasi Scrapy

Dalam mengintsall Library **Scrapy**, dapat dilakukan dengan mengetikkan code seperti dibawah ini di.

In [1]:
pip install scrapy

Note: you may need to restart the kernel to use updated packages.


# 2. Import Library

Setelah Library **Scrapy** telah berhasil di install, maka tahap selanjutnya yakni import library yang akan digunakan dalam tahapan *Crawling Data*. Dapat diketikkan seperti proses dibawah ini.

In [2]:
import scrapy
from scrapy.crawler import CrawlerProcess

# 3. Program Untuk *Crawling Data*

Tahapan selanjutnya setelah dilakukan proses Import Library yang dibutuhkan yakni melakukan tahapan *Crwaling Data*. Tahapan awal yaitu menyimpan sebuah class QuotesSpider dengan Variabel start_urls yang memiliki fungsi sebagai wadah dalam menampung target dari url. Lalu data yang akan diambil merupakan data dari Portal Tugas Akhir Univeristas Trunojoyo Madura dengan ketentuan "Ekonomi-Manajemen", data yang akan diambil merupakan dat dari halaman 1 hingga 10 yang ada di dalam protal. Prosesnya dapat dilihat seperti dibawah ini.

In [3]:
class QuotesSpider(scrapy.Spider):
    name = "quotes"
    start_urls = []
    def __init__(self):
        url = 'https://pta.trunojoyo.ac.id/welcome/index/'
        for page in range(1,10):
            self.start_urls.append(url+str(page))

    def parse(self, response):
        for detail in response.css('a.gray.button::attr(href)'): 
            yield response.follow(detail.get(), callback = self.parse_detail)

    def parse_detail(self, response):
        for data in response.css('#content_journal > ul > li'):
            yield{
                'Judul': data.css('div:nth-child(2) > a::text').get(),
                'Penulis': data.css('div:nth-child(2) > span::text').get().replace('Penulis : ', ''),
                'Dospem 1': data.css('div:nth-child(3) > span::text').get().replace('Dosen Pembimbing I : ', ''),
                'Dospem 2': data.css('div:nth-child(4) > span::text').get().replace('Dosen Pembimbing II :', ''),
                'Abstraksi': data.css('div:nth-child(2) > p::text').get().replace('\n\n|\n','').replace('ABSTRAK', ''),
                'Abstraction': data.css('div:nth-child(4) > p::text').get().replace('\n\n|\n','').replace('ABSTRACT', ''),
                'Link Download': data.css('div:nth-child(5) > a:nth-child(1)::attr(href)').get().replace('.pdf-0.jpg', '.pdf'),
            }

Jangan lupa untuk menyimpan terlebih dahulu proses yang telah dikerjakan. Kemudian buka *Command Prompt* terlebih dahulu pastikan bahwa folder yang berisikan file scrapy terbuka. Selanjutnya jalankan perintah ini pada *Command Prompt* guna memporses dan menyimpan file dalam format .csv dengan mengetikan perintah berikut ini **scrapy runspider namaFile.py -o namaFile.csv**

# Data Pre-Processing

Setelah data telah berhasil di **Crawling**. Tahapan selanjutnya yakni *Pre-Processing* Data hasil Crwaling merupakan data yang masih bersifat mentah yang perlu dilakukan tahapan *Pre-Processing* guna mendapatkan data yang clean sehingga hasil daru pemrosesan data lebih maksimal. 

Adapun tahapan dalam Pre-Processing yakni :
1. Case Folding
2. Tokenisasi
3. Stopword
4. Normalisasi
5. Stemming

# Import Library

Adapun tahap awal yang harus dilakukan sebelum tahapan *Pre-Processing* yaitu mengimport beberapa library terlebih dahulu. Library yang diimport merupakan library yang akan digunakan pada proses *Pre-Processing*.

In [4]:
import pandas as pd
import numpy as np
import string
import re #regrex libray
import nltk
import swifter
import Sastrawi

from nltk.tokenize import word_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer

# Load Dataset

Tahap selanjutnya, tidak lupa untuk melakukan Load Dataset yang akan telah di *Crawling* sebelumnya

In [5]:
data_berita = pd.read_csv('crawlUAS.csv')
data_berita.head()

,Judul,Penulis,Dospem 1,Dospem 2,Abstraksi,Abstraction,Link Download
0,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Hendri Wahyudi Prayitno,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,https://pta.trunojoyo.ac.id/uploads/journals/0...
1,Pengukuran Website Quality Pada Situs Sistem A...,Muhammad Zakaria Utomo,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,https://pta.trunojoyo.ac.id/uploads/journals/1...
2,"PENGARUH PERSEPSI MANFAAT, PERSEPSI KEMUDAHAN,...",Ahmad,"Bambang Setiyo Pambudi, S.E., MM.","Fathor AS, S.E., MM.",Tujuan penelitian ini adalah untuk mengidentif...,The purpose of this research was to identify v...,https://pta.trunojoyo.ac.id/uploads/journals/0...
3,PENGARUH PENGEMBANGAN KARIR TERHADAP SEMANGAT ...,Danial Aziz M.YD,"Dr. Hj. Iriani Ismail, Dra.,MM","Drs. Ec. Mudji Kuswinarno, M.Si",PLN merupakan Badan Usaha Milik Negara yang be...,PLN is a state-owned enterprise dealy with the...,https://pta.trunojoyo.ac.id/uploads/journals/0...
4,PENGARUH FAKTOR-FAKTOR STRES KERJA TERHADAP DI...,Saiful Rahman,"Dr. Muhammad Alkirom Wildan,S.E.,M.Si","Helmi Buyung Aulia,S.ST.,S.E.,M.MT",\r\n\r\nPenelitian ini bertujuan untuk mengeta...,\r\n\r\nThis study aimed to determine the effe...,https://pta.trunojoyo.ac.id/uploads/journals/0...


Berikut ini merupakan tampilan data yang akan dilakukan *Pre-Processing*

# 1. Case Folding

*Case folding* adalag proses pre-processing yang memiliki fungsi untuk menyeragamkan karakter pada data yang ada. Proses Case Folding merupakan proses dalam mengubah huruf kecil. Pada karakter "A-Z" menjadi "a-z"

Contoh : terdapat sebuah kata "Kecil" ketika dilakukan proses *Case Folding* maka akan berubah menjadi "kecil". Semua huruf berubah menjadi huruf kecil dengan menggunakan *lowercase* 

Prosesnya dapat dilihat seperti berikut ini :

In [6]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data_berita['Abstraksi'] = data_berita['Abstraksi'].str.lower()


print('Case Folding Result : \n')
print(data_berita['Abstraksi'].head(20))
print('\n\n\n')

Case Folding Result : 

0     abstrak\r\npenelitian ini menggunakan metode k...
1     aplikasi nyata pemanfaatan teknologi informasi...
2     tujuan penelitian ini adalah untuk mengidentif...
3     pln merupakan badan usaha milik negara yang be...
4     \r\n\r\npenelitian ini bertujuan untuk mengeta...
5     munculnya asumsi bahwa lahirnya budaya mengemi...
6     muhammad sholeh, pengaruh iklan di televisi te...
7     atribut produk memberikan pandangan atau gamba...
8     ulfiyatun mutohharoh, “analisis faktor-faktor ...
9     \r\ntujuan penelitian ini adalah untuk mengana...
10    anissa novianti. 090211100010. pengaruh atribu...
11    devi kurniawati, pengaruh earning per share, n...
12    \r\ntujuan penelitian ini adalah (1) untuk men...
13    tujuan penelitian ini adalah untuk mengetahui ...
14    dwi suhardianto (2013), “pengaruh karakteristi...
15    farid,  pengaruh  periklanan di media televisi...
16    abstrak\r\n\r\npengaruh kecerdasan emosional t...
17    \r\n\r\npenelitian

# 2. Tokenisasi

Tokenisasi merupakan salah satu tahapan yang ada dalam *Pre-Processing* dengan konsep memotong sebuah kalimat menjadi kata.

Dalam tahap tokenizing yang pertama import string dan juga mengimport library re. Dan juga diperlukan library yang digunakan untuk memproses tokenize yakni nltk.tokenize dan nltk.probability.

In [7]:
import string 
import re #regex library

# import word_tokenize & FreqDist from NLTK
from nltk.tokenize import word_tokenize 
from nltk.probability import FreqDist

def remove_tweet_special(text):
    # remove tab, new line, ans back slice
    text = text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")
    # remove non ASCII (emoticon, chinese word, .etc)
    text = text.encode('ascii', 'replace').decode('ascii')
    # remove mention, link, hashtag
    text = ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())
    # remove incomplete URL
    return text.replace("http://", " ").replace("https://", " ")
                
data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_tweet_special)

#remove number
def remove_number(text):
    return re.sub(r"\d+", "", text)

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_number)

#remove punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans("","",string.punctuation))

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_punctuation)

#remove whitespace leading & trailing
def remove_whitespace_LT(text):
    return text.strip()

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_whitespace_LT)

#remove multiple whitespace into single whitespace
def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_whitespace_multiple)

# remove single char
def remove_singl_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

data_berita['Abstraksi'] = data_berita['Abstraksi'].apply(remove_singl_char)

# NLTK word rokenize 
def word_tokenize_wrapper(text):
    return word_tokenize(text)

data_berita['Abstraksi_tokens'] = data_berita['Abstraksi'].apply(word_tokenize_wrapper)

print('Tokenizing Result : \n') 
print(data_berita['Abstraksi_tokens'].head(20))
print('\n\n\n')

Tokenizing Result : 

0     [abstrak, penelitian, ini, menggunakan, metode...
1     [aplikasi, nyata, pemanfaatan, teknologi, info...
2     [tujuan, penelitian, ini, adalah, untuk, mengi...
3     [pln, merupakan, badan, usaha, milik, negara, ...
4     [penelitian, ini, bertujuan, untuk, mengetahui...
5     [munculnya, asumsi, bahwa, lahirnya, budaya, m...
6     [muhammad, sholeh, pengaruh, iklan, di, televi...
7     [atribut, produk, memberikan, pandangan, atau,...
8     [ulfiyatun, mutohharoh, analisis, faktorfaktor...
9     [tujuan, penelitian, ini, adalah, untuk, menga...
10    [anissa, novianti, pengaruh, atribut, produk, ...
11    [devi, kurniawati, pengaruh, earning, per, sha...
12    [tujuan, penelitian, ini, adalah, untuk, menge...
13    [tujuan, penelitian, ini, adalah, untuk, menge...
14    [dwi, suhardianto, pengaruh, karakteristik, in...
15    [farid, pengaruh, periklanan, di, media, telev...
16    [abstrak, pengaruh, kecerdasan, emosional, ter...
17    [penelitian, ini, be

# 3. Stopword

*Stopword* merupakan tahapan mengambil kata-kata yang penting dari hasil token dengan menggunakan algoritma stoplist (membuang kata yang dinilai kurang penting) atau bisa disebit worldlist (menyimpan kata yang penting). 

Contoh dari stopword dalam bahasa indonesia yakni sepert "dan","di","dari" dan masih banyak lainnya.

*Stopword* memiliki additional stopword yang dilist. Jadi ada beberapa kata yang tidak diperlukan dalam data. Kata tersebut dimuat dalam list_stopwords.exted Kemudian dibuat sebuah fungsi dengan nama stopword_removal yang akan membuang kata yang tidak penting di dalam atribut "Abstraksi"

In [8]:
from nltk.corpus import stopwords
# ----------------------- get stopword from NLTK stopword -------------------------------
# get stopword indonesia
list_stopwords = stopwords.words('indonesian')


# ---------------------------- manualy add stopword  ------------------------------------
# append additional stopword
list_stopwords.extend(["yg", "dg", "rt", "dgn", "ny", "d", 'klo', 
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 
                       'gak', 'ga', 'krn', 'nya', 'nih', 'sih', 
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 
                       'nyg', 'hehe', 'pen', 'u', 'nan', 'loh', 'rt',
                       '&amp', 'yah'])

# ----------------------- add stopword from txt file ------------------------------------
# read txt stopword using pandas
txt_stopword = pd.read_csv("crawlUAS.csv", names= ["stopwords"], header = None)

# convert stopword string to list & append additional stopword
list_stopwords.extend(txt_stopword["stopwords"][0].split(' '))

# ---------------------------------------------------------------------------------------

# convert list to dictionary
list_stopwords = set(list_stopwords)


#remove stopword pada list token
def stopwords_removal(words):
    return [word for word in words if word not in list_stopwords]

data_berita['Abstraksi_tokens_WSW'] = data_berita['Abstraksi_tokens'].apply(stopwords_removal) 


print(data_berita['Abstraksi_tokens_WSW'].head(20))

0     [abstrak, penelitian, metode, kuantitatif, men...
1     [aplikasi, nyata, pemanfaatan, teknologi, info...
2     [tujuan, penelitian, mengidentifikasi, variabe...
3     [pln, badan, usaha, milik, negara, bergerak, b...
4     [penelitian, bertujuan, pengaruh, faktorfaktor...
5     [munculnya, asumsi, lahirnya, budaya, mengemis...
6     [muhammad, sholeh, pengaruh, iklan, televisi, ...
7     [atribut, produk, pandangan, gambaran, produk,...
8     [ulfiyatun, mutohharoh, analisis, faktorfaktor...
9     [tujuan, penelitian, menganalisis, atribut, ku...
10    [anissa, novianti, pengaruh, atribut, produk, ...
11    [devi, kurniawati, pengaruh, earning, share, n...
12    [tujuan, penelitian, pengaruh, pesan, iklan, t...
13    [tujuan, penelitian, pengaruh, variabel, brand...
14    [dwi, suhardianto, pengaruh, karakteristik, in...
15    [farid, pengaruh, periklanan, media, televisi,...
16    [abstrak, pengaruh, kecerdasan, emosional, pre...
17    [penelitian, bertujuan, pengaruh, citra, m

# 4. Normalisasi

Normalisasi merupakan salah satu tahapan yang ada dalam pre-processing. Normalisasi memiliki fungsi untuk menyamakan nilai yang ada dalam data dalam rentan tertentu.

In [9]:
normalizad_word = pd.read_csv("crawlUAS.csv")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
    if row[0] not in normalizad_word_dict:
        normalizad_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

data_berita['Abstraksi_normalized'] = data_berita['Abstraksi_tokens_WSW'].apply(normalized_term)

data_berita['Abstraksi_normalized'].head(20)

0     [abstrak, penelitian, metode, kuantitatif, men...
1     [aplikasi, nyata, pemanfaatan, teknologi, info...
2     [tujuan, penelitian, mengidentifikasi, variabe...
3     [pln, badan, usaha, milik, negara, bergerak, b...
4     [penelitian, bertujuan, pengaruh, faktorfaktor...
5     [munculnya, asumsi, lahirnya, budaya, mengemis...
6     [muhammad, sholeh, pengaruh, iklan, televisi, ...
7     [atribut, produk, pandangan, gambaran, produk,...
8     [ulfiyatun, mutohharoh, analisis, faktorfaktor...
9     [tujuan, penelitian, menganalisis, atribut, ku...
10    [anissa, novianti, pengaruh, atribut, produk, ...
11    [devi, kurniawati, pengaruh, earning, share, n...
12    [tujuan, penelitian, pengaruh, pesan, iklan, t...
13    [tujuan, penelitian, pengaruh, variabel, brand...
14    [dwi, suhardianto, pengaruh, karakteristik, in...
15    [farid, pengaruh, periklanan, media, televisi,...
16    [abstrak, pengaruh, kecerdasan, emosional, pre...
17    [penelitian, bertujuan, pengaruh, citra, m

# 5. Stemming

*Stemming* adalah salah satu tahap yang ada dalam *Pre-Processing*, *Stemming* memiliki fungsu untuk mengubah kata dalam bentuk yang baku atau dasar. Dalam python *Stemming* menggunakan library *Sastrawi*.

*Stemming* ini tidak digunakan dalam topic modeliing.

In [10]:
# import Sastrawi package
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter


# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stemmed
def stemmed_wrapper(term):
    return stemmer.stem(term)

term_dict = {}

for document in data_berita['Abstraksi_normalized']:
    for term in document:
        if term not in term_dict:
            term_dict[term] = ' '
            
print(len(term_dict))
print("------------------------")

for term in term_dict:
    term_dict[term] = stemmed_wrapper(term)
    print(term,":" ,term_dict[term])
    
print(term_dict)
print("------------------------")


# apply stemmed term to dataframe
def get_stemmed_term(document):
    return [term_dict[term] for term in document]

data_berita['Abstraksi_tokens_stemmed'] = data_berita['Abstraksi_normalized'].swifter.apply(get_stemmed_term)
print(data_berita['Abstraksi_tokens_stemmed'])

7548
------------------------
abstrak : abstrak
penelitian : teliti
metode : metode
kuantitatif : kuantitatif
menekankan : tekan
pengujian : uji
hipotesis : hipotesis
variabel : variabel
kepemimpinanx : kepemimpinanx
kompensasi : kompensasi
dilaksanakan : laksana
kantor : kantor
dinas : dinas
pendidikan : didik
kabupaten : kabupaten
sampang : sampang
sampel : sampel
orang : orang
pegawai : pegawai
negeri : negeri
sipil : sipil
non : non
pns : pns
bebas : bebas
bersamasama : bersamasama
hubungan : hubung
signifikan : signifikan
simultan : simultan
kepemimpinan : pimpin
kinerja : kerja
terbukti : bukti
nilai : nilai
fhitung : fhitung
tingkat : tingkat
pengaruh : pengaruh
dominan : dominan
hitung : hitung
positif : positif
parsial : parsial
terikat : ikat
kesimpulan : simpul
berdasarkan : dasar
statistik : statistik
memiliki : milik
membuktikan : bukti
berpengaruh : pengaruh
kunci : kunci
aplikasi : aplikasi
nyata : nyata
pemanfaatan : manfaat
teknologi : teknologi
informasi : informasi
k

quota : quota
dianalisis : analis
jalur : jalur
path : path
usahanya : usaha
kamalia : kamalia
marketing : marketing
mix : mix
sentra : sentra
mutira : mutira
batik : batik
meliputi : liput
lokasi : lokasi
penelitiannya : teliti
mutiara : mutiara
survey : survey
bebasnya : bebas
terikatnya : ikat
probability : probability
realibilitas : realibilitas
men : men
guji : guji
varabel : varabel
squere : squere
variasi : variasi
diluar : luar
keempat : empat
berdasrkan : berdasrkan
achmad : achmad
sulaiman : sulaiman
keuangan : uang
kpri : kpri
eka : eka
karsa : karsa
tinjau : tinjau
aspek : aspek
perputaran : putar
piutang : piutang
kas : kas
profitabilitasdibawah : profitabilitasdibawah
chairul : chairul
anam : anam
purnamawati : purnamawati
bagaiman : bagaiman
diskriptif : diskriptif
langkahlangkah : langkahlangkah
mnghitung : mnghitung
inventesmen : inventesmen
membutuhkan : butuh
peraturan : atur
mentri : tri
koperasi : koperasi
umkm : umkm
efisien : efisien
ayu : ayu
massita : massita
e

businees : businees
association : association
kiddrock : kiddrock
distro : distro
cranberries : cranberries
survei : survei
lapangan : lapang
field : field
pengolaha : pengolaha
identifikasi : identifikasi
kesediaan : sedia
menerima : terima
perluasan : luas
merekomendasikan : rekomendasi
cranberriess : cranberriess
apresisasikan : apresisasikan
presepsi : presepsi
satiyah : satiyah
kelautan : laut
perikanan : ikan
drahjsanugrahini : drahjsanugrahini
irawatimm : irawatimm
auliasstsemmt : auliasstsemmt
mudah : mudah
instansi : instansi
efektif : efektif
pengembnagan : pengembnagan
menyesuaikan : sesuai
kebutuhankebutuhan : kebutuhankebutuhan
pengetahuan : tahu
tuntutan : tuntut
mendukung : dukung
terciptanya : cipta
kondusif : kondusif
produktivitasi : produktivitas
menganalisa : menganalisa
observasional : observasional
analitik : analitik
kuisioner : kuisioner
probality : probality
jabatan : jabat
partisipasi : partisipasi
seleksi : seleksi
peserta : serta
instruktur : instruktur
kebe

nama : nama
melekat : lekat
ingatan : ingat
dikeluarkan : keluar
berpengraruh : berpengraruh
angket : angket
dps : dps
pemilihan : pilih
eksternal : eksternal
persediaan : sedia
bahan : bahan
baku : baku
propindo : propindo
wira : wira
proyek : proyek
carson : carson
regency : regency
economic : economic
order : order
quantity : quantity
reorder : reorder
eoq : eoq
pemesanan : mesan
pembangunan : bangun
pasir : pasir
zak : zak
biji : biji
bata : bata
putih : putih
pesanan : pesan
rit : rit
pengaman : kam
penghematan : hemat
experiential : experiential
sense : sense
feel : feel
think : think
act : act
relate : relate
kepuasaan : puasa
yuliana : yuliana
index : index
jii : jii
penanaman : tanam
dikemudian : kemudi
penilaian : nilai
diinvestasikan : investasi
single : single
pembentukan : bentuk
kreteria : kreteria
alam : alam
sutera : sutera
realty : realty
asri : asri
unilever : unilever
unvr : unvr
smgr : smgr
lippo : lippo
karawaci : karawaci
lpkr : lpkr
kalbe : kalbe
farma : farma
kl

silmultan : silmultan
dependennya : dependen
reni : reni
strata : strata
forensik : forensik
selling : selling
direct : direct
jenuh : jenuh
marti : marti
ekowatianalisis : ekowatianalisis
glagah : glagah
kabupatan : kabupatan
lamonganskripsi : lamonganskripsi
dosenpembimbing : dosenpembimbing
idr : idr
transakasi : transakasi
purposivesampling : purposivesampling
fitriya : fitriya
insentif : insentif
ro : ro
raden : raden
mas : mas
mengikutsertakan : mengikutsertakan
luas : luas
pengikut : ikut
interprestasi : interprestasi
peristiwaperistiwa : peristiwaperistiwa
pengikutnya : ikut
pengorganisasian : organisasi
aktivitasaktivitas : aktivitasaktivitas
memelihara : pelihara
perolehan : oleh
orangorang : orangorang
verbal : verbal
akrab : akrab
sosialnya : sosial
kehadiran : hadir
halhal : halhal
penerimanya : terima
niaga : niaga
tama : tama
raharja : raharja
disproportionate : disproportionate
berstrata : strata
ajeng : ajeng
sri : sri
jayanti : jayanti
swasta : swasta
bopo : bopo
siya

...

pangeranpersero : pangeranpersero
pangeran : pangeran
mufid : mufid
hidayatulloh : hidayatulloh
aliyuddin : aliyuddin
asstsemmt : asstsemmt
menjadikannya : jadi
perpaduan : padu
angketkuesioner : angketkuesioner
ftest : ftest
diketaui : tau
jabatanx : jabatanx
ybaik : ybaik
malihat : malihat
kepuasankerja : kepuasankerja
yberpengaruh : yberpengaruh
thitungttabel : thitungttabel
variabelyang : variabelyang
kuncirotasi : kuncirotasi
jabatankepuasan : jabatankepuasan
pegawaikinerja : pegawaikinerja
kependidikananalisis : kependidikananalisis
nia : nia
rohmatin : rohmatin
nikmah : nikmah
bc : bc
sophie : sophie
paris : paris
ridha : ridha
faridiawati : faridiawati
kepecayaan : kepecayaan
mengontrol : kontrol
reation : reation
pmii : pmii
arno : arno
mengekplorasi : mengekplorasi
rayon : rayon
asshiddiq : asshiddiq
ikamapaba : ikamapaba
istrumen : istrumen
backup : backup
ditarik : tarik
mempertahan : tahan
anny : anny
pangistu : pangistu
ning : ning
pensiun : pensiun
ptpelindo : ptpelindo


komensasi : komensasi
requested : requested
amount : amount
sharing : sharing
estimate : estimate
peer : peer
lending : lending
ammana : ammana
farida : farida
esate : esate
devinta : devinta
dianasari : dianasari
sisahasil : sisahasil
rpublik : rpublik
danibu : danibu
menyajikan : saji
rab : rab
proposive : proposive
crdebt : crdebt
pemaegang : pemaegang
sholichul : sholichul
amin : amin
mike : mike
ardita : ardita
kabupeten : kabupeten
robiatul : robiatul
adawiyah : adawiyah
rientasi : rientasi
arosbaya : arosbaya
memepengaruhi : pengaruh
tehnikstratifiedclustersamplingberdasarkan : tehnikstratifiedclustersamplingberdasarkan
bsebesar : bsebesar
betakepemimpinan : betakepemimpinan
besi : besi
unggul : unggul
eksplanatori : eksplanatori
arana : arana
prasarana : prasarana
excellentia : excellentia
lawangan : lawang
chika : chika
hanifzainuri : hanifzainuri
pengaruhinformasilaba : pengaruhinformasilaba
pertumbuhanlabaterhadapkoefisienresponlabapada : pertumbuhanlabaterhadapkoefisienresp

ulum : ulum
abel : abel
ditoko : toko
slovinanalisis : slovinanalisis
bresponden : bresponden
berdasaekan : berdasaekan
fani : fani
rachma : rachma
noerlila : noerlila
isa : isa
anshori : anshori
fanomena : fanomena
menyayangkan : sayang
tempuh : tempuh
destination : destination
jaddih : jaddih
tersebar : sebar
menunjukkanbahwasannya : menunjukkanbahwasannya
kelamahan : kelamahan
itemitem : itemitem
holistik : holistik
penggambaran : gambar
emosi : emosi
membangkitkan : bangkit
santai : santai
waswas : waswas
tenang : tenang
keamanannya : aman
distribution : distribution
diproduksi : produksi
katun : katun
logo : logo
memperbarui : baru
assfak : assfak
assyuhada : assyuhada
tema : tema
kekinian : kini
mengangkat : angkat
moral : moral
diindikasikan : indikasi
sandal : sandal
samplingyaitu : samplingyaitu
sedudo : sedudo
assistance : assistance
others : others
expressing : expressing
feelings : feelings
incentives : incentives
helping : helping
company : company
syaicullah : syaicullah


Pandas Apply:   0%|          | 0/1019 [00:00<?, ?it/s]

0       [abstrak, teliti, metode, kuantitatif, tekan, ...
1       [aplikasi, nyata, manfaat, teknologi, informas...
2       [tuju, teliti, identifikasi, variabelvariabel,...
3       [pln, badan, usaha, milik, negara, gerak, bida...
4       [teliti, tuju, pengaruh, faktorfaktor, stres, ...
                              ...                        
1014    [umi, kholidah, pengaruh, arus, kas, operasi, ...
1015    [abstrak, ra, fajria, nurul, islami, pengaruh,...
1016    [riska, utami, ningsih, pengaruh, milik, insti...
1017    [dasar, hasil, teliti, nilai, square, besar, p...
1018    [laura, monica, analisis, pengaruh, likuiditas...
Name: Abstraksi_tokens_stemmed, Length: 1019, dtype: object


# Topik Modeling LSA (Latent Semantic Analysis)

Topik Modeling LSA (Latent Semantic Analysis) adalah sebuah teknik yang digunakan untuk menentukan sebuah tema yang ada pada dokumen. 

Tahapan dalam Topik Modeling LSA (Latent Semantic Analysis), yakni :
1. Pre-Processing
2. Menghitung TF-IDF
3. Menampilkan SVD
4. Topic dan Term

# 1. Pre-Processing

# Import Library

Seperti pada tahapan *Pre-processing* pada umumnya, dalam LSA juga terdapat *Pre-processing*. Tahap awal yakni Import Library yang akan digunakan.

In [11]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#import nltk
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

#preprocessing
from nltk.corpus import stopwords  #stopwords
from nltk import word_tokenize,sent_tokenize # tokenizing
from nltk.stem import PorterStemmer,LancasterStemmer  # using the Porter Stemmer and Lancaster Stemmer and others
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer  # lammatizer from WordNet

# for named entity recognition (NER)
from nltk import ne_chunk

# vectorizers for creating the document-term-matrix (DTM)
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

#stop-words
stop_words=set(nltk.corpus.stopwords.words('english'))

# Load Dataset

Muat data yang akan dilakukan Topic Modeling

In [12]:
data_berita = pd.read_csv('crawlUAS.csv')
data_berita.head()

,Judul,Penulis,Dospem 1,Dospem 2,Abstraksi,Abstraction,Link Download
0,PENGARUH KEPEMIMPINAN DAN KOMPENSASI TERHADAP ...,Hendri Wahyudi Prayitno,"Dra. Hj. S Anugrahini Irawati, MM","Helmi Buyung Aulia,S,ST,SE,.MT",Abstrak\r\nPenelitian ini menggunakan metode k...,Abstract\r\nThis research use quantitative met...,https://pta.trunojoyo.ac.id/uploads/journals/0...
1,Pengukuran Website Quality Pada Situs Sistem A...,Muhammad Zakaria Utomo,"Dr. Ir. Nurita Andriani, MM","Nirma Kurriwati, SP, M.Si",Aplikasi nyata pemanfaatan teknologi informasi...,Academic portal system in University of Trunoj...,https://pta.trunojoyo.ac.id/uploads/journals/1...
2,"PENGARUH PERSEPSI MANFAAT, PERSEPSI KEMUDAHAN,...",Ahmad,"Bambang Setiyo Pambudi, S.E., MM.","Fathor AS, S.E., MM.",Tujuan penelitian ini adalah untuk mengidentif...,The purpose of this research was to identify v...,https://pta.trunojoyo.ac.id/uploads/journals/0...
3,PENGARUH PENGEMBANGAN KARIR TERHADAP SEMANGAT ...,Danial Aziz M.YD,"Dr. Hj. Iriani Ismail, Dra.,MM","Drs. Ec. Mudji Kuswinarno, M.Si",PLN merupakan Badan Usaha Milik Negara yang be...,PLN is a state-owned enterprise dealy with the...,https://pta.trunojoyo.ac.id/uploads/journals/0...
4,PENGARUH FAKTOR-FAKTOR STRES KERJA TERHADAP DI...,Saiful Rahman,"Dr. Muhammad Alkirom Wildan,S.E.,M.Si","Helmi Buyung Aulia,S.ST.,S.E.,M.MT",\r\n\r\nPenelitian ini bertujuan untuk mengeta...,\r\n\r\nThis study aimed to determine the effe...,https://pta.trunojoyo.ac.id/uploads/journals/0...


# Case Folding

Case folding adalag proses pre-processing yang memiliki fungsi untuk menyeragamkan karakter pada data yang ada. Proses Case Folding merupakan proses dalam mengubah huruf kecil. Pada karakter "A-Z" menjadi "a-z"

Dalam LSA juga diperlukan tahapan *Case Folding*

In [13]:
# ------ Case Folding --------
# gunakan fungsi Series.str.lower() pada Pandas
data_berita['Abstraksi'] = data_berita['Abstraksi'].str.lower()


print('Case Folding Result : \n')
print(data_berita['Abstraksi'].head(20))
print('\n\n\n')

Case Folding Result : 

0     abstrak\r\npenelitian ini menggunakan metode k...
1     aplikasi nyata pemanfaatan teknologi informasi...
2     tujuan penelitian ini adalah untuk mengidentif...
3     pln merupakan badan usaha milik negara yang be...
4     \r\n\r\npenelitian ini bertujuan untuk mengeta...
5     munculnya asumsi bahwa lahirnya budaya mengemi...
6     muhammad sholeh, pengaruh iklan di televisi te...
7     atribut produk memberikan pandangan atau gamba...
8     ulfiyatun mutohharoh, “analisis faktor-faktor ...
9     \r\ntujuan penelitian ini adalah untuk mengana...
10    anissa novianti. 090211100010. pengaruh atribu...
11    devi kurniawati, pengaruh earning per share, n...
12    \r\ntujuan penelitian ini adalah (1) untuk men...
13    tujuan penelitian ini adalah untuk mengetahui ...
14    dwi suhardianto (2013), “pengaruh karakteristi...
15    farid,  pengaruh  periklanan di media televisi...
16    abstrak\r\n\r\npengaruh kecerdasan emosional t...
17    \r\n\r\npenelitian

Sebelum beralih pada tahap selanjutnya, ada beberapa atribut yang perlu dihapus yakni judul, penulis, dospem 1, dospem 2, abstaction, dan link download. Dikarenakan atribut yang digunakan yakni atribut "Abstraksi"

In [14]:
# drop judul,penulis, dospem 1 & 2
data_berita.drop(['Judul', 'Penulis', 'Dospem 1', 'Dospem 2','Abstraction', 'Link Download'],axis=1,inplace=True)
data_berita.head()

,Abstraksi
0,abstrak\r\npenelitian ini menggunakan metode k...
1,aplikasi nyata pemanfaatan teknologi informasi...
2,tujuan penelitian ini adalah untuk mengidentif...
3,pln merupakan badan usaha milik negara yang be...
4,\r\n\r\npenelitian ini bertujuan untuk mengeta...


Sehingga menyisakan atribut Abstraksi saja

# Data Cleaning

Pada tahap selnajutnya yakni melakukan cleaning dan juga tahap pre-processing kembali. Menggunakan lemmatizer dan juga stemmer. Terdapat juga kata-kata yang tidak digubakan bersama dan juga kata-kata yang panjanya kurang dari 3 karakter yang berguna untuk mengurangi beberapa kata yang tidak cocok.

In [15]:
def clean_text(headline):
  le=WordNetLemmatizer()
  word_tokens=word_tokenize(headline)
  tokens=[le.lemmatize(w) for w in word_tokens if w not in stop_words and len(w)>3]
  cleaned_text=" ".join(tokens)
  return cleaned_text

In [16]:
# time taking
data_berita['Abstraksi_cleaned_text']=data_berita['Abstraksi'].apply(clean_text)

In [17]:
data_berita.head()

,Abstraksi,Abstraksi_cleaned_text
0,abstrak\r\npenelitian ini menggunakan metode k...,abstrak penelitian menggunakan metode kuantita...
1,aplikasi nyata pemanfaatan teknologi informasi...,aplikasi nyata pemanfaatan teknologi informasi...
2,tujuan penelitian ini adalah untuk mengidentif...,tujuan penelitian adalah untuk mengidentifikas...
3,pln merupakan badan usaha milik negara yang be...,merupakan badan usaha milik negara yang berger...
4,\r\n\r\npenelitian ini bertujuan untuk mengeta...,penelitian bertujuan untuk mengetahui pengaruh...


# 2. Menghitung TF-IDF

Tahap pertama dalam Topic Modelling LSA yakni menghitung TF-IDF. 

SA pada umumnua diimplementasikan dengan menggunakan nilai TF-Idf dan tidak dengan Count Vectorizer max_feature bergantung pada daya komputasi dan juga pada eval. Metrik merupakan skor yang koheren untuk menentukan model Nilai default untuk min_df dan max_df dapat bekerja dengan baik Dapat mencoba nilai yang berbeda-beda dalam ngram_range

In [19]:
vect =TfidfVectorizer(stop_words=list_stopwords,max_features=1000) 
vect_text=vect.fit_transform(data_berita['Abstraksi'])
print(vect_text.shape)
print(vect_text)

c:\users\asus\appdata\local\programs\python\python38\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'download', 'kali', 'kurangnya', 'link', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


(1019, 1000)
  (0, 495)	0.02413291326466229
  (0, 146)	0.026344440548850632
  (0, 550)	0.08696458625102069
  (0, 553)	0.05272593095848641
  (0, 889)	0.07444263561005116
  (0, 140)	0.03617371136448918
  (0, 458)	0.07469980823139569
  (0, 933)	0.13219453054233427
  (0, 646)	0.034205209434297223
  (0, 749)	0.03815800524292645
  (0, 344)	0.07867177844845882
  (0, 259)	0.09269712948806874
  (0, 686)	0.07911283376740323
  (0, 789)	0.06752412090053751
  (0, 8)	0.08743090131854725
  (0, 944)	0.04894975464109875
  (0, 311)	0.06679870044330434
  (0, 611)	0.1113619459021283
  (0, 931)	0.20522768114712567
  (0, 466)	0.1324620765659283
  (0, 865)	0.0673495395377575
  (0, 860)	0.08339063334852218
  (0, 348)	0.07775581686328156
  (0, 130)	0.170950371161862
  (0, 615)	0.07631196182014573
  :	:
  (1018, 878)	0.0474678693359781
  (1018, 961)	0.06601600939224292
  (1018, 607)	0.1424036080079343
  (1018, 141)	0.01577229806590107
  (1018, 514)	0.018037661423300922
  (1018, 796)	0.0147564468956091
  (1018, 

In [20]:
idf=vect.idf_

In [21]:
dd=dict(zip(vect.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])
print(dd['penelitian'])
print(dd['need'])  # police is most common and forecast is least common among the news headlines.

penelitian need
1.0068864020296335
7.234410725718371


Oleh karena itu, berdasarkan pada nilai idf yang ada 'penelitian' adalah kata yang paling sering muncul sedangkan 'need' paling jarang muncul dalam berita

# 3. Menampilkan SVD

Setelah berhasil dalam menghitung TF-IDFnya. Tahapan selanjutnya yakni menampilkan SVDnya.

SVD merupakan sebuah proses yang memiliki fungsi untuk menemukan sebuah informasi yang memiliki nilai yang penting guna menampilkan nilai yang sama.

- Berikut ini merupakan rumus dari SVD :
                              M=U * S * V
                              
- Rumus dalam SVD untuk mebgitung matrix U dan V
                               A=U S V^{T}

Berikut ini proses menampilan SVD :

In [22]:
from sklearn.decomposition import TruncatedSVD
lsa_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=42)

lsa_top=lsa_model.fit_transform(vect_text)

In [23]:
print(lsa_top)
print(lsa_top.shape)  # (no_of_doc*no_of_topics)

[[ 0.32584027 -0.05616913  0.03591451 ... -0.13171864  0.1205522
  -0.0172367 ]
 [ 0.1192496   0.00588017  0.0902221  ...  0.09780151 -0.1333223
   0.08155446]
 [ 0.1185041   0.0347926   0.08335458 ...  0.02957189 -0.00504653
   0.02570972]
 ...
 [ 0.20265561  0.20579079 -0.07557669 ... -0.08558212 -0.01619141
  -0.0251432 ]
 [ 0.29472214  0.30423748 -0.1115254  ... -0.14522478 -0.03592332
  -0.04928105]
 [ 0.19290203  0.18389485 -0.08865498 ...  0.48844365  0.10398064
  -0.13570719]]
(1019, 10)


In [24]:
l=lsa_top[0]
print("Document 0 :")
for i,topic in enumerate(l):
  print("Topic ",i," : ",topic*100)

Document 0 :
Topic  0  :  32.584026599424895
Topic  1  :  -5.616913385213299
Topic  2  :  3.591450737135063
Topic  3  :  -1.6220794290807774
Topic  4  :  -19.04733028928685
Topic  5  :  -5.987325118539571
Topic  6  :  22.16274571616939
Topic  7  :  -13.171863909407655
Topic  8  :  12.055220344725058
Topic  9  :  -1.72367044202152


Hampir sama dengan dokumen lainnya, dapat dilakukan proses seperti di bawah ini. Akan tetapi perlu diperhatikan bahwa dalam setiap nilai tidak menambah 1 itu bukan sebuah kemungkinan topik yang ada dalam dokumen.

In [25]:
print(lsa_model.components_.shape) # (no_of_topics*no_of_words)
print(lsa_model.components_)

(10, 1000)
[[ 0.0632527   0.00993873  0.00773284 ...  0.00541176  0.00382589
   0.00748169]
 [-0.03293959 -0.00141886 -0.00171204 ...  0.0063977   0.00730271
   0.01628051]
 [-0.01695686 -0.0033747  -0.00114528 ...  0.01545939 -0.00414587
  -0.01032939]
 ...
 [-0.03672922 -0.00453417 -0.00088816 ... -0.00199009  0.03044559
   0.00971479]
 [-0.00585226 -0.00350361  0.0016267  ...  0.00462474  0.00436617
   0.00116986]
 [ 0.01141344 -0.00246094 -0.00166343 ... -0.00426934  0.00205731
   0.00516926]]


# 4. Topic dan Term

Tahapan terakhir dalam LSA yakni menentukan Topic dan Term dalam setiap dokumen. 

Seperti dibawah ini, dilakukan looping sebanyak 10 kali dan menghasilkan 10 topic

In [26]:
# most important words for each topic
vocab = vect.get_feature_names()

for i, comp in enumerate(lsa_model.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")

Topic 0: 
kerja karyawan kinerja variabel penelitian perusahaan berpengaruh nilai signifikan uji 

Topic 1: 
perusahaan ratio saham return equity harga keputusan bursa efek terdaftar 

Topic 2: 
keputusan pembelian produk merek konsumen kualitas promosi citra variabel uji 

Topic 3: 
ratio return equity saham to on harga debt earning current 

Topic 4: 
kerja kepuasan prestasi stres kualitas produktivitas struktur pelayanan modal perusahaan 

Topic 5: 
saham harga split stock volume perdagangan perbedaan return portofolio pt 

Topic 6: 
kepuasan pegawai kepemimpinan pelayanan pelanggan kualitas organisasi gaya komitmen kabupaten 

Topic 7: 
bank tbk rasio pt keuangan loan ldr eva npl kepuasan 

Topic 8: 
pegawai kerja prestasi disiplin bank promosi kepemimpinan kabupaten motivasi dinas 

Topic 9: 
merek citra mahasiswa brand kepemimpinan trunojoyo universitas gaya pegawai madura 

